# Summary of extrinsic analysis

This notebook produced a quick overview of core results from the extrinsic results.

Only run this notebook once you have used a reference data set and successfully run the notebook for the [extrinsic analysis](./extrinsinc_analysis.ipynb).

For further information on how to use and interpret the different metrics, we refer to the original analysis notebook.

In [63]:
import geopandas as gpd
import osmnx as ox
import networkx as nx
import yaml
import matplotlib.pyplot as plt
import contextily as cx
import json
import pickle
import pandas as pd 
import numpy as np
import os.path
from src import evaluation_functions as ef
from src import matching_functions as mf

In [3]:
with open(r'../config.yml') as file:

    parsed_yaml_file = yaml.load(file, Loader=yaml.FullLoader)

    study_area = parsed_yaml_file['study_area']
    study_area_poly_fp = parsed_yaml_file['study_area_poly']
    study_crs = parsed_yaml_file['study_crs']

    missing_tag_dict = parsed_yaml_file['missing_tag_analysis']

    incompatible_tags_dict = parsed_yaml_file['incompatible_tags_analysis']

    cycling_infrastructure_queries = parsed_yaml_file['cycling_infrastructure_queries']

    reference_geometries = parsed_yaml_file['reference_geometries']
    cycling_bidirectional = parsed_yaml_file['bidirectional']

    feature_matching = parsed_yaml_file['feature_matching']
    
print('Settings loaded!')

Settings loaded!


**Load data:**

In [40]:
osm_graph = ox.load_graphml(f'../data/osm_{study_area}.graphml')
osm_simplified_graph = ox.load_graphml(f'../data/osm_{study_area}_simple.graphml')

# Convert to nodes and edges
osm_nodes, osm_edges = ox.graph_to_gdfs(osm_graph)
osm_simplified_nodes, osm_simplified_edges = ox.graph_to_gdfs(osm_simplified_graph)

# Fix converted boolean values
osm_simplified_edges.loc[osm_simplified_edges.cycling_bidirectional=='True','cycling_bidirectional'] = True
osm_simplified_edges.loc[osm_simplified_edges.cycling_bidirectional=='False','cycling_bidirectional'] = False

# Load simplified and non-simplified graphs
ref_graph = ox.load_graphml(f'../data/ref_{study_area}.graphml')
ref_simplified_graph = ox.load_graphml(f'../data/ref_{study_area}_simple.graphml')

# Convert to nodes and ref_edges
ref_nodes, ref_edges = ox.graph_to_gdfs(ref_graph)
ref_simplified_nodes, ref_simplified_edges = ox.graph_to_gdfs(ref_simplified_graph)


print('Data loaded!')

Data loaded!


**Load results:**

In [43]:
with open(f'../results/extrinsic_analysis_{study_area}.json') as input_file:
    all_results = json.load(input_file)

with open(f'../results/grid_results_extrinsic_{study_area}.pickle', 'rb') as fp:
    grid = pickle.load(fp)

with open(f'../results/feature_matches_{study_area}.json') as input_file:
    fm_results = json.load(input_file)

with open(f'../results/grid_results_feature_matching_{study_area}.pickle', 'rb') as fp:
    grid_fm = pickle.load(fp)

In [49]:
osm_df = pd.DataFrame.from_dict(all_results['osm_results'], orient='index')
ref_df = pd.DataFrame.from_dict(all_results['ref_results'], orient='index')

osm_df.rename(columns={0:'OSM'},inplace=True)
ref_df.rename(columns={0:'reference'}, inplace=True)

combined_results = pd.concat([osm_df, ref_df], axis=1)

combined_results = combined_results.round(2)

In [64]:
combined_results

,OSM,reference
node_count,4690.00,3655.00
edge_count,5437.00,4208.00
edge_density,5847.17,3437.55
node_density,5847.17,3437.55
danling_node_density,9.79,4.81
protected_density,5282.10,2996.63
unprotected_density,514.23,440.92
mixed_density,50.67,NaN
simplified_edge_pct_diff,82.43,60.18
simplified_node_pct_diff,84.47,63.50


In [66]:
# Set caption

# Colors to index

# Change column font to something bigger

# Color largest and smallest value in each row

In [8]:
grid

,grid_id,geometry,count_osm_edges,count_osm_nodes,count_osm_simplified_edges,count_osm_simplified_nodes,count_ref_edges,count_ref_nodes,count_ref_simplified_edges,count_ref_simplified_nodes,...,ref_node_edge_ratio,component_ids_osm,component_ids_ref,cells_reached_osm,cells_reached_ref,cell_reach_diff,count_osm_dangling_nodes,count_ref_dangling_nodes,osm_dangling_nodes_per_node,ref_dangling_nodes_per_node
0,0,"POLYGON ((710193.940 6181853.300, 710682.960 6...",46.0,43.0,14.0,11.0,NaN,NaN,NaN,NaN,...,NaN,[0],,653,0,653,5.0,NaN,0.454545,NaN
1,1,"POLYGON ((710682.960 6181371.592, 710682.960 6...",38.0,35.0,12.0,9.0,NaN,NaN,NaN,NaN,...,NaN,[0],,653,0,653,5.0,NaN,0.555556,NaN
2,2,"POLYGON ((710682.960 6180889.885, 710682.960 6...",21.0,20.0,4.0,3.0,2.0,2.0,2.0,2.0,...,1.0,[0],[139],653,7,646,2.0,2.0,0.666667,1.0
3,3,"POLYGON ((710682.960 6180408.177, 710682.960 6...",46.0,45.0,20.0,19.0,1.0,1.0,1.0,1.0,...,1.0,"[0, 243]",[139],655,7,648,11.0,1.0,0.578947,1.0
4,4,"POLYGON ((710682.960 6179926.469, 710682.960 6...",31.0,31.0,15.0,15.0,1.0,1.0,1.0,1.0,...,1.0,"[0, 243]",[137],655,9,646,10.0,1.0,0.666667,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,765,"POLYGON ((727309.640 6173664.269, 727309.640 6...",18.0,17.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,[0],,653,0,653,NaN,NaN,NaN,NaN
766,766,"POLYGON ((727309.640 6173182.561, 727309.640 6...",10.0,10.0,3.0,3.0,NaN,NaN,NaN,NaN,...,NaN,"[0, 293]",,654,0,654,3.0,NaN,1.000000,NaN
767,767,"POLYGON ((727309.640 6172700.853, 727309.640 6...",16.0,17.0,3.0,4.0,NaN,NaN,NaN,NaN,...,NaN,"[0, 292]",,654,0,654,3.0,NaN,0.750000,NaN
768,768,"POLYGON ((727309.640 6172219.145, 727309.640 6...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,,0,0,0,NaN,NaN,NaN,NaN


In [ ]:
# Read dictionary into dataframe with result type as index and osm/ref as columns

# Compute new col with difference

# Plot difference based on color? Or plot all values based on color?

## Local differences

In [ ]:
# List of values to be plotted and labels

# Create subplots based on lengths

# Flatten axes?

# Delete unneeded plot

# Plot values

# Set axis off

# How to control colors?


# Plot of differences in:
#  network density length
# Node density
# Protected density
# Unprotected density
# Local node/edge ratio



# Dangling node density 


# All this is saved to grid - no need for recomputing anything

In [ ]:
# Also load feature matching results and add to things to plot!

## Component comparison

In [ ]:
# Plots of dangling nodes per grid cell for both
# Plots of cc and cc_i for both

# Plot of connected component connectivity

In [ ]:
# Export results

**How to summarize feature matching?**